Get dataset and requirement

In [17]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jkk-0YM2S8MEezIbjduptU6tMXgHzPOs' -O assin2-blind-test.xml
!wget http://nilc.icmc.usp.br/assin/assin.tar.gz
!tar -xzf assin.tar.gz
%rm -rf assin.tar.gz

!pip install xmltodict

%mkdir $download_folder
%mv *.xml $download_folder

--2020-05-04 23:31:38--  https://docs.google.com/uc?export=download&id=1jkk-0YM2S8MEezIbjduptU6tMXgHzPOs
Resolving docs.google.com (docs.google.com)... 172.217.204.100, 172.217.204.102, 172.217.204.113, ...
Connecting to docs.google.com (docs.google.com)|172.217.204.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jmumksdspv5cgnhk0u0jmnk94qdcf3tf/1588635075000/06286219297811011113/*/1jkk-0YM2S8MEezIbjduptU6tMXgHzPOs?e=download [following]
--2020-05-04 23:31:38--  https://doc-14-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jmumksdspv5cgnhk0u0jmnk94qdcf3tf/1588635075000/06286219297811011113/*/1jkk-0YM2S8MEezIbjduptU6tMXgHzPOs?e=download
Resolving doc-14-3g-docs.googleusercontent.com (doc-14-3g-docs.googleusercontent.com)... 172.217.203.132, 2607:f8b0:400c:c07::84
Connecting to doc-14-3g-docs.googleusercontent.com (doc-14

In [0]:
dataset = 'assin-1+2'

##### get file from mt-dnn_port repo
mode = 'st-dnn'
#'st-dnn', 'mt-dnn_assin2', 'mt-dnn_assin-ptbr+assin2','mt-dnn_assin', 'mt-dnn_assin+tweetsent'
pretrained = 'bert-pt_base'

rte_filepath = \
f'output/{mode}/{dataset}-rte/{pretrained}/{dataset}-rte_test_scores_4.json' if mode == 'st-dnn' \
else  f'output/{mode}/{pretrained}/{dataset}-rte_test_scores_4.json'

sts_filepath = \
f'output/{mode}/{dataset}-sts/{pretrained}/{dataset}-sts_test_scores_4.json' if mode == 'st-dnn' \
else  f'output/{mode}/{pretrained}/{dataset}-sts_test_scores_4.json'
########


### change to other filepath here
filepaths = {
    'rte': rte_filepath,
    'sts': sts_filepath,
}

output_dir = f'report/{mode}/{pre-trained}'
download_folder = 'data/dataset'

Generate XML output

In [0]:
from os import path, mkdir
import xmltodict
import json

tasks = ['rte', 'sts']
scores = dict()

if dataset == 'assin-1+2':
    corpora = ['assin-ptbr', 'assin-ptpt', 'assin2']
elif dataset == 'assin-ptbr+2':
    corpora = ['assin-ptbr', 'assin2']
else:
    corpora = [dataset]

for corpus in corpora:
    for task in tasks:
        filepath = filepaths[task]
        with open(filepath) as f:
            scores[task] = json.load(f)

        goldfile = 'assin2-blind-test.xml' if corpus == 'assin2' else f'{corpus}-test.xml'
        with open(f'{download_folder}/{goldfile}') as f:
            xml = xmltodict.parse(f.read())

    for idx, item in enumerate(xml['entailment-corpus']['pair']):
        uid = xml['entailment-corpus']['pair'][idx]['@id']
        if len(corpora) >= 2: #uses data augmentation
            displacement = {
                'assin-ptbr': 10000,
                'assin-ptpt': 20000,
                'assin2': 30000,
            }
            uid = str(int(uid) + displacement[corpus])

        pos = scores['rte']['uids'].index(uid)

        similarity = scores['sts']['scores'][pos]
        entailment_labels = ['Entailment','None','Paraphrase']
        entailment = entailment_labels[scores['rte']['predictions'][pos]]

        xml['entailment-corpus']['pair'][idx]['@similarity'] = round(similarity, 1)
        xml['entailment-corpus']['pair'][idx]['@entailment'] = entailment

    result = xmltodict.unparse(xml, pretty = True)
    if len(corpora) == 1:
        outpath = f'{output_dir}/{corpus}-test.xml'
    else: #uses data augmentation
        if not path.exists(f'{output_dir}/{dataset}'):
            mkdir(f'{output_dir}/{dataset}')
        outpath = f'{output_dir}/{dataset}/{corpus}-test.xml'

    with open(outpath, 'w') as f:
        f.write(result)